# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — найти ответы на четыре вопроса:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?


Входные данные от банка — статистика о платёжеспособности клиентов.

**Ход исследования**

Данные о клиентах мы получили из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед поиском ответов понадобится обзор данных. 

Проверим данные на ошибки, и оценим их влияние на исследование. 

 
Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Поиск ответов на вопросы.


## Обзор данных

Составим первое представление о данных банка.

In [ ]:
from pymystem3 import Mystem
import pandas as pd

In [ ]:
df = pd.read_csv('/datasets/data.csv')

Выведем на экран первые десять строк таблицы:

In [ ]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. Тип данных в двух столбцах `float64`, в пяти `int64`, во всех остальных столбцах — `object`.

Согласно документации к данным:
*	`children` — количество детей в семье
*	`days_employed` — общий трудовой стаж в днях
*	`dob_years` — возраст клиента в годах
*	`education` — уровень образования клиента
*	`education_id` — идентификатор уровня образования
*	`family_status` — семейное положение
*	`family_status_id` — идентификатор семейного положения
*	`gender` — пол клиента
*	`income_type` — тип занятости
*	`debt` — имел ли задолженность по возврату кредитов
*	`total_income` — ежемесячный доход
*	`purpose` — цель получения кредита


Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Посмотрим какие значения присутствуют в столбцах:

In [ ]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Похоже, что при заполнении этой графы сотрудники банка ошиблись: -1 ребёнок это скорее всего 1 ребёнок, а 20 детей это скорее всего 2 ребёнка. Просто случайно нажали дополнительно клавишу и не заметили. Заменим значения согласно этой теории.

In [ ]:
df.loc[df['children'] == -1, 'children'] = 1

In [ ]:
df.loc[df['children'] == 20, 'children'] = 2

Проверим, что теперь количество детей похоже на правду:

In [ ]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Изучим столбец с возрастом:

In [ ]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце в возрастом присутствуют нули... Скорее всего оператор не нажал цифру стоящую перед нулём и это люди с юбилеем (20,40,60 лет). Для поставленных вопросов возраст не нужен, поэтому ничего для данной задачи с этими нулями делать не будем.

Изучим столбец уровня образования:

In [ ]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [ ]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Похоже, что категорий образования сильно меньше, чем значений в столбце "образование". Исправим эту ситуацию.

In [ ]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Изучим столбец семейного положения:

In [ ]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [ ]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

С семейным положением и индификатором положения всё в порядке.

Изучим столбец пола клиента:

In [ ]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Одному человеку не указали его пол, но для поставленных задач это не важно.

Изучим столбец с типом занятости:

In [ ]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Никаких подозрительных типов занятости не обнаружено.

Изучим столбец по задолженности кредитов:

In [ ]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Со столбцом о задолженности кредитов всё в порядке. 1 - имел задолженность, 0 - не имел.

**Вывод**
Данные нуждаются в предобработке, нужно поработать с пропусками и категоризацией данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем, сколько и в каких столбцах пропущеные значения:

In [ ]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в days_employed (трудовой стаж) пропуски не повлияют на поиск ответов.

Но пропуски в total_income (ежемесячный доход) могут помешать определить зависимость между уровнем дохода и возвратом кредита в срок. 

Количество пропусков в двух столбцах совпадают, проверим, одинаковые ли это строки или нет.




In [ ]:
df.loc[df['total_income'].isna()]['days_employed'].unique()  #.value_counts()

array([nan])

В строках, где в столбце `total_income` пропущенное значение, в `days_employed` значения пропущены тоже.

Пропуски в количественных значениях заполняют характерными значениями. Это значения, характеризующие состояние выборки.
Просто взять медиану или среднее по всему столбцу будет некорректно.

Будем опираться на данные из другого столбца. Выберем столбец `income_type` - тип занятости.

Найдём медианные значения дохода в зависимости от типа занятости людей:



In [ ]:
#Создадим новый столбец, в котором указаны медианные значения дохода для каждой строки согласно типу занятости
median_total_income = df.groupby('income_type')['total_income'].transform('mean')
median_total_income

0        161380.260488
1        161380.260488
2        161380.260488
3        161380.260488
4        137127.465690
             ...      
21520    202417.461462
21521    137127.465690
21522    161380.260488
21523    161380.260488
21524    161380.260488
Name: total_income, Length: 21525, dtype: float64

In [ ]:
# заполним все пустые значения дохода на медианные значения из нового столбца
df['total_income'] = df['total_income'].fillna(median_total_income)

Теперь разберёмся с трудовым стажем в днях, посмотрим, какие встречаются значения:

In [ ]:
df['days_employed'].head(15)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
Name: days_employed, dtype: float64

Трудовой стаж не может быть отрицательным, скорее всего программа его так указывает по своим каким-то причинам. Переведём все отрицательные числа в положительные.

In [ ]:
df['days_employed'] = df['days_employed'].abs() #все отрицательные значения сделаем положительными

Также прикинем какой трудовой стаж может быть у взрослого человека: если пойти на работу в 18 лет и работать 60 лет (до возраста 78 лет), в году примерно 237 рабочих дней, то получается 14220 рабочих дней у взрослого человека.
В серии встречаются значения сильно больше этого числа. Предположение, что данные записаны в часах тоже не оправданы: 340266/8=42533, что тоже очень много. Посмотрим, сколько таких значений.

In [ ]:
237*60

14220

In [ ]:
340266/8

42533.25

In [ ]:
df[df['days_employed'] > 15000]['days_employed'].count()

3458

Всего 3458 явно больших значений. Заменим в этих строках рабочий стаж на значения, описанные формулой `(age-18)*273`.

In [ ]:
days_employed_avg = df.groupby('income_type')['dob_years'].transform(lambda x: (x - 18)*273) 
#создадим Series с расчётными значениями рабочих дней для каждого человека в зависимости от возраста человека

In [ ]:
df.loc[df['days_employed'] > 15000, 'days_employed'] = days_employed_avg
#если количество рабочих дней больше 15000, заменим значение на расчитанное

In [ ]:
df['days_employed'].head(15)

0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4     9555.000000
5      926.185831
6     2879.202052
7      152.779569
8     6929.865299
9     2188.756445
10    4171.483647
11     792.701887
12            NaN
13    1846.641941
14    1844.956182
Name: days_employed, dtype: float64

У нас остались пропуски в этом столбце:

In [ ]:
df['days_employed'].isna().sum()

2174

Аналогично замене в столбце `total_income`, проведём замену на медианные значения, опираясь на данные из  столбца `income_type` - тип занятости.

Найдём медианные трудового стажа в зависимости от типа занятости людей и заполним ими пропуски:


In [ ]:
#Создадим новый столбец, в котором указаны медианные значения дохода для каждой строки согласно типу занятости
median_days_employed = df.groupby('income_type')['days_employed'].transform('mean')
# заполним все пустые значения стажа на медианные значения из нового столбца
df['days_employed'] = df['days_employed'].fillna(median_days_employed)

Убедимся, что в таблице не осталось пропусков.

In [ ]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Строки, в которых не был указан ежемесячный доход совпали со строками, где не был указан трудовой стаж клиентов. Вероятно, это или сбой в системе, или клиенты по каким-то причинам не сообщили эти данные.

Пропуски в ежемесячном доходе и трудовом стаже заменены на медианные значения согласно типу занятости людей. 
Некорректно большие значения в трудовом стаже заменены на условно-допустимые. 
Теперь в данных нет пропущеных значений.

### Замена типа данных

Столбцы `days_employed` и `total_income` представлены вещественным типом данных, но трудовой стаж в днях может быть только целым числом. Ежемесячный доход может быть вещественным, но если отбросить копейки, можно получить удобные для работы целые числа. На результаты данное преобразование не повлияет.

Заменим вещественный тип данных на целочисленный. Поскольку столбцы уже представлены вещественными числами (и в них нет пропусков), просто изменим их тип методом `.astype()`:



In [ ]:
df['total_income'] = df['total_income'].astype('int')

In [ ]:
df['days_employed'] = df['days_employed'].astype('int')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Теперь в таблице всего два типа данных - целые числа и объекты.

### Обработка дубликатов

Посчитаем явные дубликаты в таблице:

In [ ]:
df.duplicated().sum()

71

Возможно, это сбой системы, при котором создались дубликаты. Удалим явные дубликаты и переобозначим индексы заново:

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.duplicated().sum()

0

**Вывод**

Явные дубликаты выявлены и удалены. Провести поиск неявных дубликатов не вижу возможности.

### Лемматизация

Выделим леммы в столбце с целями получения кредита: для этого импортируем библиотеку для лемматизации и создадим функцию, которая будет лемматизировать текст. потом передадим эту функцию методу .apply() для столбца с целями кредита, получим новый столбец со списками лемм. 

In [ ]:
m = Mystem()

# лемматизируем столбец "цель кредита"
df['lemmas_purpose'] = df.purpose.apply(m.lemmatize)
df['lemmas_purpose']

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: lemmas_purpose, Length: 21454, dtype: object

**Вывод**

Создали в ДатаФрейме дополнительный столбец, куда лемматизировали цели кредита.
Лемматизацию строк провели с помощью созданной функции.

### Категоризация данных

Чтобы ответить на поставленные вопросы, необходимо ввести категории.

- *Есть ли зависимость между наличием детей и возвратом кредита в срок?*
Выделим категории "нет детей", "1-2 ребёнка в семье", "многодетная семья".

- *Есть ли зависимость между семейным положением и возвратом кредита в срок?*
Категории для данного вопроса это идентификатор семейного положения `family_status_id`.

- *Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*
Категории для данного вопроса явно не определены, их необходимо выбрать. 
Пусть будет три категории: низкий, средний и высокий доход.
За средний доход примем медиану выборки `total_income` и +-50% от выборки. До этого низкий доход, после - высокий.

- *Как разные цели кредита влияют на его возврат в срок?*
Здесь проведём исследование и выберем категории, в итоге будет что-то вроде "свадьба", "машина", "образование".

**Проведём категоризацию по количеству детей.**

In [ ]:
def children_category(children):
    if children == 0:
        return 'нет детей'
    if 1 <= children <= 2:
        return '1-2 ребёнка в семье'
    return 'многодетная семья'

In [ ]:
df['children_category'] = df['children'].apply(children_category)

Количество строк в каждой категории:

In [ ]:
df['children_category'].value_counts()

нет детей              14091
1-2 ребёнка в семье     6983
многодетная семья        380
Name: children_category, dtype: int64

**Проведём категоризацию по уровню дохода.** Проанализируем столбец `total_income`.

In [ ]:
# проведём границы для среднего дохода. В первом приближении разделять будем выборку по уровню 25% и 75% от количества людей.
# Чуть ниже посмотрим, что получится.
total_income_low_level = int(df['total_income'].count()/4)    # нижняя граница выборки (в количестве людей)
total_income_high_level = int(df['total_income'].count()*3/4) # верхняя граница выборки (в количестве людей)

In [ ]:
total_income_sort = sorted(df['total_income']) # отсортируем выборку по возрастанию

In [ ]:
# заработок для граничных условий:
total_income_sort[total_income_low_level]   
total_income_sort[total_income_high_level]   

202417

In [ ]:
pd.DataFrame({
    'min'            : [df['total_income'].min()],
    'max'            : [df['total_income'].max()],
    'mean'           : [int(df['total_income'].mean())],
    'low_level_mean' : total_income_sort[total_income_low_level] ,
    'high_level_mean': total_income_sort[total_income_high_level],
    'low_level'      : [total_income_low_level],
    'high_level'     : [total_income_high_level],
    'count'          : [df['total_income'].count()]
})

,min,max,mean,low_level_mean,high_level_mean,low_level,high_level,count
0,20667,2265604,167431,107620,202417,5363,16090,21454


Получили таблицу расчётных значений для нашей гипотезы.
Мне кажется удачным выбор такой категоризации. Я бы округлила до 100 и 200 тысяч, чтобы было удобно разговаривать о этих числах.
Тогда все значения разделим на три категории:
- Низкий доход все значения до 100т.р.
- Средний доход от 100т.р до 200т.р. включительно
- Высокий доход все значения больше 200т.р.

Напишем и применим функцию для категоризации дохода.

In [ ]:
def income_category(total_income):
    if total_income < 100000:
        return 'низкий'
    if 100000 <= total_income <= 300000:
        return 'средний'
    return 'высокий'

In [ ]:
df['total_income_category'] = df['total_income'].apply(income_category)

Количество строк в каждой категории:

In [ ]:
df.groupby('total_income_category')['total_income_category'].count()

total_income_category
высокий     1483
низкий      4463
средний    15508
Name: total_income_category, dtype: int64

**Проведём категоризацию по целям кредита.** Проанализируем столбец `purpose` и `lemmas_purpose`.

In [ ]:
df['purpose'].head(20)

0                              покупка жилья
1                    приобретение автомобиля
2                              покупка жилья
3                 дополнительное образование
4                            сыграть свадьбу
5                              покупка жилья
6                          операции с жильем
7                                образование
8                      на проведение свадьбы
9                    покупка жилья для семьи
10                      покупка недвижимости
11         покупка коммерческой недвижимости
12                           сыграть свадьбу
13                   приобретение автомобиля
14                покупка жилой недвижимости
15    строительство собственной недвижимости
16                              недвижимость
17                строительство недвижимости
18        на покупку подержанного автомобиля
19              на покупку своего автомобиля
Name: purpose, dtype: object

In [ ]:
df['lemmas_purpose'].head(20)

0                               [покупка,  , жилье, \n]
1                     [приобретение,  , автомобиль, \n]
2                               [покупка,  , жилье, \n]
3                  [дополнительный,  , образование, \n]
4                             [сыграть,  , свадьба, \n]
5                               [покупка,  , жилье, \n]
6                        [операция,  , с,  , жилье, \n]
7                                     [образование, \n]
8                   [на,  , проведение,  , свадьба, \n]
9             [покупка,  , жилье,  , для,  , семья, \n]
10                       [покупка,  , недвижимость, \n]
11      [покупка,  , коммерческий,  , недвижимость, \n]
12                            [сыграть,  , свадьба, \n]
13                    [приобретение,  , автомобиль, \n]
14             [покупка,  , жилой,  , недвижимость, \n]
15    [строительство,  , собственный,  , недвижимост...
16                                   [недвижимость, \n]
17                 [строительство,  , недвижимос

Считаю возможным выделить следующие категории:
- Покупка автомобиля
- Покупка недвижимости
- Строительство недвижимости
- Проведение свадьбы
- Образование

Создадим функцию, ищущую категорию из списка:

In [ ]:
def purpose_category(lemmas_purpose):
    if (('жилье' in lemmas_purpose) or ('недвижимость' in lemmas_purpose)) and ('строительство' in lemmas_purpose):
        return 'строительство недвижимости'
    if ('жилье' in lemmas_purpose) or ('недвижимость' in lemmas_purpose):
        return 'покупка недвижимости'
    if 'автомобиль' in lemmas_purpose:
        return 'покупка автомобиля'
    if 'образование' in lemmas_purpose:
        return 'образование'
    if 'свадьба' in lemmas_purpose:
        return 'свадьба'
    return lemmas_purpose

In [ ]:
purpose_category(df['lemmas_purpose'][10]) #здесь можно проверить адекватную работу функции

'покупка недвижимости'

In [ ]:
df['purpose_category'] = df['lemmas_purpose'].apply(purpose_category)

Количество строк в каждой категории:

In [ ]:
df['purpose_category'].value_counts()

покупка недвижимости          8933
покупка автомобиля            4306
образование                   4013
свадьба                       2324
строительство недвижимости    1878
Name: purpose_category, dtype: int64

**Проведём категоризацию по наличию долгов.**

In [ ]:
def debt_category(debt):
    if debt == 1:
        return 'Долги были'
    return 'Долгов не было'

In [ ]:
df['debt_category'] = df['debt'].apply(debt_category)

**Вывод**

Ввели категории:
- Наличие детей:
    * нет детей
    * 1-2 ребёнка в семье
    * многодетная семья
- Семейное положение:
    * женат / замужем
    * гражданский брак
    * Не женат / не замужем
    * в разводе
    * вдовец / вдова
- Уровень дохода:
    * низкий
    * средний
    * высокий
- Цели кредита:
    * покупка недвижимости
    * строительство недвижимости
    * покупка автомобиля
    * свадьба
    * образование
- Наличие задолженностей:
    * Долги были
    * Долгов не было

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
children_data_pivot = df.pivot_table(
    index='children_category', 
    columns='debt_category', 
    values='debt', 
    aggfunc='count'
)

In [ ]:
children_data_pivot['ratio, %'] = children_data_pivot['Долги были'] / children_data_pivot['Долгов не было'] *100

In [ ]:
children_data_pivot.sort_values(by='ratio, %', ascending=False)

debt_category,Долги были,Долгов не было,"ratio, %"
children_category,,,
1-2 ребёнка в семье,647,6336,10.211490
многодетная семья,31,349,8.882521
нет детей,1063,13028,8.159349


**Вывод**

In [ ]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,children_category,total_income_category,purpose_category,debt_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",1-2 ребёнка в семье,средний,покупка недвижимости,Долгов не было
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",1-2 ребёнка в семье,средний,покупка автомобиля,Долгов не было
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",нет детей,средний,покупка недвижимости,Долгов не было
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",многодетная семья,средний,образование,Долгов не было
4,0,9555,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",нет детей,средний,свадьба,Долгов не было
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",нет детей,средний,покупка недвижимости,Долгов не было
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",нет детей,средний,покупка недвижимости,Долгов не было
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",нет детей,средний,образование,Долгов не было
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",1-2 ребёнка в семье,низкий,свадьба,Долгов не было
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",нет детей,средний,покупка недвижимости,Долгов не было


Люди с одним-двумя детьми отдают кредит хуже, чем люди не имеющие детей или многодетные.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_data_pivot = df.pivot_table(
    index='family_status', 
    columns='debt_category', 
    values='debt', 
    aggfunc='count'
)

In [ ]:
family_data_pivot['ratio, %'] = family_data_pivot['Долги были'] / family_data_pivot['Долгов не было'] *100

In [ ]:
family_data_pivot.sort_values(by='ratio, %', ascending=False)

debt_category,Долги были,Долгов не было,"ratio, %"
family_status,,,
Не женат / не замужем,274,2536,10.804416
гражданский брак,388,3763,10.310922
женат / замужем,931,11408,8.160940
в разводе,85,1110,7.657658
вдовец / вдова,63,896,7.031250


**Вывод**

Люди, не имеющие пары, хуже всех отдают кредит. Между людьми живущими гражданским браком и людьми с официально оформленым браком различие более двух процентов при чём официально расписанные люди отдают кредит лучше.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_data_pivot = df.pivot_table(
    index='total_income_category', 
    columns='debt_category', 
    values='debt', 
    aggfunc='count'
)

In [ ]:
income_data_pivot['ratio, %'] = income_data_pivot['Долги были'] / income_data_pivot['Долгов не было'] *100

In [ ]:
income_data_pivot.sort_values(by='ratio, %', ascending=False)

debt_category,Долги были,Долгов не было,"ratio, %"
total_income_category,,,
средний,1281,14227,9.004006
низкий,354,4109,8.615235
высокий,106,1377,7.697894


**Вывод**

Люди со среднем доходом отдают кредит хуже, чем люди с высоким доходом. Люди с низким доходом отдают кредит чуть лучше, чем люди со среднем доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_data_pivot = df.pivot_table(
    index='purpose_category', 
    columns='debt_category', 
    values='debt', 
    aggfunc='count'
)

In [ ]:
purpose_data_pivot['ratio, %'] = purpose_data_pivot['Долги были'] / purpose_data_pivot['Долгов не было'] *100

In [ ]:
purpose_data_pivot.sort_values(by='ratio, %', ascending=False)

debt_category,Долги были,Долгов не было,"ratio, %"
purpose_category,,,
покупка автомобиля,403,3903,10.325391
образование,370,3643,10.156464
свадьба,186,2138,8.699719
строительство недвижимости,144,1734,8.304498
покупка недвижимости,638,8295,7.691380


**Вывод**

Цели кредита влияют на его возврат в срок. Лучше всего возвращают кредит взятый на покупку и строительство недвижимости, хуже всего возвращают кредит за покупку автомобиля и за получение образования.

## Шаг 4. Общий вывод

Мы нашли ответы на вопросы и установили что действиельно: количество детей, семейное положение человека, его доход и цель кредита влияют на возврат кредита в срок.

1. Люди с одним-двумя детьми отдают кредит хуже, чем люди не имеющие детей или многодетные.

2. Люди, не имеющие пары, хуже всех отдают кредит. Между людьми живущими гражданским браком и людьми с официально оформленым браком различие более двух процентов при чём официально расписанные люди отдают кредит лучше.

3. Люди со среднем доходом отдают кредит хуже, чем люди с высоким доходом. Люди с низким доходом отдают кредит чуть лучше, чем люди со среднем доходом.

4. Цели кредита влияют на его возврат в срок. Лучше всего возвращают кредит взятый на покупку и строительство недвижимости, хуже всего возвращают кредит за покупку автомобиля и за получение образования.


Из данных одного банка не всегда можно сделать корректный вывод о всех кредитуемых клиентах. Необходимо проверять достоверность результатов с помощью статистических гипотез, но пока что проект этого не требует.